- BERT모델에 문장을 입력해서 벡터로 변환하기

In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 582 kB 47.9 MB/s 
     |████████████████████████████████| 2.8 MB 43.1 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 76.7 MB/s 
     |████████████████████████████████| 419 kB 78.1 MB/s 
     |████████████████████████████████| 596 kB 76.7 MB/s 
     |████████████████████████████████| 880 kB 48.7 MB/s 
     |████████████████████████████████| 3.3 MB 51.4 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 1.1 MB 53.6 MB/s 
     |████████████████████████████████| 96 kB 6.3 MB/s 
     |████████████████████████████████| 271 kB 64.8 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 81.6 MB/s 
  Created wheel for sacremoses: filenam

### 토크나이저 선언

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

## 모델 초기화
-> Bert model이 pretrain 할 때 썼던 토크나이저를 그대로 사용하여야 벡터변환에 문제가 없음 !  
- 모델과 토크나이저의 토큰화 방식이 다르면 결과가 이상하게 나옴

In [ ]:
from transformers import BertConfig, BertModel

#pretrained_model_config 에는 BERT모델을 프리트레인할때 설정했던 내용 그대로
pretrained_model_config = BertConfig.from_pretrained(   
    "beomi/kcbert-base"
)

#모델 전체를 초기화 한후 미리 학습된 kcbert 체크포인트를 불러들임
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config = pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# BERT 모델을 프리트레인할 때 설정했던 내용
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

### 입력값 만들기

In [ ]:
sentences = ["안녕하십니까","하이요","펭하!"]

features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation = True,
)

In [ ]:
features

{'input_ids': [[2, 19017, 22796, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 4040, 3, 0, 0, 0, 0, 0, 0], [2, 3279, 4159, 5, 3, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}

attention_mask -> 패딩인 토큰이면 0, 패딩이 아닌 토큰이면 1

In [ ]:
tokenizer(sentences)

{'input_ids': [[2, 19017, 22796, 3], [2, 15830, 4040, 3], [2, 3279, 4159, 5, 3]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1]]}

### BERT로 단어/문장 수준의 벡터 구하기
- 피쳐를 토치 텐서로 변환

In [ ]:
features.items()

dict_items([('input_ids', [[2, 19017, 22796, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 4040, 3, 0, 0, 0, 0, 0, 0], [2, 3279, 4159, 5, 3, 0, 0, 0, 0, 0]]), ('token_type_ids', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), ('attention_mask', [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])])

In [ ]:
for k, v in features.items():
  print(k,torch.tensor(v))

input_ids tensor([[    2, 19017, 22796,     3,     0,     0,     0,     0,     0,     0],
        [    2, 15830,  4040,     3,     0,     0,     0,     0,     0,     0],
        [    2,  3279,  4159,     5,     3,     0,     0,     0,     0,     0]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
dicts= {}
for k, v in features.items():
  dicts[k] = k
  dicts[v] = torch.tensor(v)
  #print(k,torch.tensor(v))
dicts 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


{'attention_mask': 'attention_mask',
 'input_ids': 'input_ids',
 tensor([[    2, 19017, 22796,     3,     0,     0,     0,     0,     0,     0],
         [    2, 15830,  4040,     3,     0,     0,     0,     0,     0,     0],
         [    2,  3279,  4159,     5,     3,     0,     0,     0,     0,     0]]): tensor([[    2, 19017, 22796,     3,     0,     0,     0,     0,     0,     0],
         [    2, 15830,  4040,     3,     0,     0,     0,     0,     0,     0],
         [    2,  3279,  4159,     5,     3,     0,     0,     0,     0,     0]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]): tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]): tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0,

In [ ]:
import torch
#딕셔너리 형태로 받아옴
features = {k: torch.tensor(v) for k,v in features.items()}
features

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


{'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[    2, 19017, 22796,     3,     0,     0,     0,     0,     0,     0],
         [    2, 15830,  4040,     3,     0,     0,     0,     0,     0,     0],
         [    2,  3279,  4159,     5,     3,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
outputs = model(**features)

In [ ]:
model(features)

AttributeError: ignored

In [ ]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.4433, -0.9240,  1.7883,  ..., -0.8659,  0.9727,  0.9249],
                                                        [-1.6224, -0.0682,  0.1769,  ..., -0.8680,  0.7786,  1.9937],
                                                        [-1.1414, -0.1960, -0.7010,  ...,  0.6878,  0.3851,  0.4984],
                                                        ...,
                                                        [-1.0095,  0.0397,  1.5963,  ...,  0.8457,  0.3166, -0.0534],
                                                        [-1.2240,  0.4880,  1.3206,  ...,  1.4305,  0.7170,  0.7261],
                                                        [-1.2351, -0.0513,  1.4715,  ...,  0.6764,  0.1578, -0.1697]],
                                               
                                                       [[ 1.0656, -0.6992,  1.4739,  ..., -0.9266,  0.5746,  0.276

In [ ]:
# **단어수준 임베딩**
print(outputs.last_hidden_state.shape)
print(outputs.last_hidden_state)

torch.Size([3, 10, 768])
tensor([[[-0.4433, -0.9240,  1.7883,  ..., -0.8659,  0.9727,  0.9249],
         [-1.6224, -0.0682,  0.1769,  ..., -0.8680,  0.7786,  1.9937],
         [-1.1414, -0.1960, -0.7010,  ...,  0.6878,  0.3851,  0.4984],
         ...,
         [-1.0095,  0.0397,  1.5963,  ...,  0.8457,  0.3166, -0.0534],
         [-1.2240,  0.4880,  1.3206,  ...,  1.4305,  0.7170,  0.7261],
         [-1.2351, -0.0513,  1.4715,  ...,  0.6764,  0.1578, -0.1697]],

        [[ 1.0656, -0.6992,  1.4739,  ..., -0.9266,  0.5746,  0.2763],
         [ 1.1682, -0.2893,  0.2161,  ..., -0.8393, -0.9610, -0.6130],
         [ 0.2997, -1.2743, -0.0026,  ...,  0.5456,  0.8371,  1.2898],
         ...,
         [ 1.4404, -0.0126, -0.3952,  ...,  0.5461,  1.2216,  0.3210],
         [ 0.4486, -1.2183,  0.9212,  ...,  1.0384,  1.6450,  0.8763],
         [ 1.6208, -0.1239, -0.3191,  ...,  0.5550,  1.4743,  0.1471]],

        [[ 0.4644, -1.4644, -0.4885,  ...,  0.2780,  0.6166,  0.2251],
         [ 0.0258, -

`torch.Size([3,10,768])`  
문장 세개에 속한 각각의 토큰을 768차원짜리 벡터로 변환했다는 것  
- 이 tensor output은 입력 단어 각각에 해당하는 BERT의 마지막 레이어 출력 벡터들

In [ ]:
# **문장 수준 임베딩**
print(outputs.pooler_output)
print(outputs.pooler_output.shape)

tensor([[ 0.1988,  0.0735,  0.0183,  ...,  0.2965, -0.1216, -0.9984],
        [-0.8787,  0.1829,  0.3193,  ...,  0.4408,  0.0386, -0.9914],
        [-0.8202,  0.0593, -0.1897,  ..., -0.3348,  0.1729, -0.9776]],
       grad_fn=<TanhBackward0>)
torch.Size([3, 768])


- 문장 3개가 768차원 벡터로 변환됐다는 의미